# Die Kommandosubstitution und *Dinge* finden

## Was behandeln wir in diesem Notebook / Lernziele

<div class="alert alert-success">
<ul>
    <li> Was ist die <i>Kommandosubstitution</i> und wie benutze ich sie?</li>
    <li> Wie finde ich bestimmte Dateien und Verzeichnisse?</li>
    <li> Wie finde ich bestimmte Dateiinhalte in Textdateien?</li>
</ul>

<b> Zeitaufwand für diese Lektion: </b> 
<ul>
    <li> Durcharbeiten des Textes: 30 min</li>
    <li> Verständnisfragen: 20 min</li>
</ul>
</div>

## Die Kommandosubstitution

Bisher haben wir längere Listen aus Dateien für Kommandos oder Shellskripte effektiv mit Wildcards erstellt. Die Wildcards `*` und `?` erlauben es uns, mit sehr wenig Tippaufwand lange Dateilisten nach Mustern in den Dateinamen zu erstellen. Oft möchte man jedoch Dateilisten nach völlig anderen Kriterien zusammenstellen, z.B aufgrund des Dateiinhalts. Eine Möglichkeit, solche Listen ähnlich effizient wie mit Wildcards zu erzeugen, ist die so-genannte *Kommandosubstitution*.
Sehen wir uns ein konkretes Beispiel an, in dem Annika die Integrität ihrer Beobachtungen intensiver testen möchte: 

In [ ]:
cd ~/Bachelor_Arbeit/Beobachtungen/
ls -F

Nachdem Annika, wie in [Lektion 5](05_Shell_for_Schleife.ipynb#Statistik_Beobachtungen), Statistiken aller Beobachtungen berechnet hat, möchte sie weitergehende Tests durchführen. Sie weiß, dass jede der ungefähr 500 Dateien in den Ordnern `raw` und `processed` genau 100 Zeilen enthalten sollte. Sie testet zunächst diese Annahme:

In [ ]:
# Teste, ob alle Dateien in Verzeichnis raw genau 100 Zeilen haben
wc -l raw/*.txt

Das sieht beim schnellen Überfliegen gut aus. Für einen kompakteren Überblick bei den vielen Dateien schreibt Annika aber noch eine erweiterte Pipeline, die nur anzeigt, wieviele Dateien es mit wievielen Zeilen gibt:

In [ ]:
# Teste, ob alle Dateien in Verzeichnis raw genau 100 Zeilen haben
wc -l raw/*.txt | awk '{print $1}' | sort | uniq -c

Die Ausgabe bedeutet, dass es 481 Dateien mit je 100 Zeilen gibt. Die zweite Zeile der Ausgabe, `1 48100`, kommt von der Gesamtbilanz, die `wc` bei der Eingabe mehrerer Dateien zusätzlich angibt. Nun dasselbe für den `processed`-Ordner:

In [ ]:
wc -l processed/*.txt | awk '{print $1}' | sort | uniq -c

Hier gibt es offenbar ein Problem und wir haben Dateien mit 100, 95 und 90 Zeilen. Annika möchte zunächst die Dateien mit 95 Zeilen genauer untersuchen und sie hierzu in einen separaten Ordner kopieren. Sie erstellt sich zunächst eine Liste der fraglichen Dateien:

In [ ]:
# liste alle Dateien in processed, die genau 95 Zeilen haben
wc -l processed/*.txt | awk '($1 == 95) {print $2}'

Mit der Syntax `awk '($1 == 95) {print $2}'` listet `awk` die zweite Spalte aller Einträge, die in der ersten Spalte einen numerischen Wert von 95 haben. Diese Liste sichern wir in einer Datei:

In [ ]:
wc -l processed/*.txt | awk '($1 == 95) {print $2}' > problem_95.txt
cat problem_95.txt

Nun kopieren wir die in `problem_95.txt` gelisteten Dateien in ein Verzeichnis `problemfaelle`:

In [ ]:
mkdir problemfaelle

In [ ]:
cp $(cat problem_95.txt) problemfaelle
ls problemfaelle

Bei dem `cp $(cat problem_95.txt) problemfaelle`-Befehl passiert folgendens: Sieht die Shell in einem Befehl das Konstrukt `$(...)`, so wird zunächst der darin eingebettete Befehl ausgeführt. Danach wird das gesamte Konstrukt `$(...)` durch *die Ausgabe dieses Befehls ersetzt* und die Abarbeitung der Kommandozeile fortgesetzt.

Im vorliegenden Fall liefert `cat problem_95.txt` die Ausgabe `processed/695833p.txt ... processed/774957p.txt`. Hiermit wird für den Befehl `cp $(cat problem_95.txt) problemfaelle` nach der *Kommandosubstitution* effektiv der Befehl `cp processed/695833p.txt ... processed/774957p.txt problemfaelle` ausgeführt. Dies ist genau die gewünschte Kopieroperation.

Die Kommandosubstitution verhält sich damit ähnlich der Behandlung eines Befehls mit den Wildcards `*` und `?`. Somit können wir hier effektiv *eigene* Wildcards mit *beliebigen* Befehlen erzeugen!

Genau, wie wir z.B. in [Lektion 5](05_Shell_for_Schleife.ipynb#for-statistik) Wildcards für Listen in `for`-Schleifen verwendet haben, können wir dies auch mit der Kommandosubstitution tun. Annika könnte daher folgenden Befehl verwenden, um Statistiken nur von den problematischen Daten zu errechnen:

In [ ]:
# Berechne Staistik für problematische Felder,
# die in 'problem_95.txt' gelistet sind:
for BEOBACHTUNG in $(cat problem_95.txt)
do
  echo "Ich arbeite an ${BEOBACHTUNG}"
  python3 calc_stats.py ${BEOBACHTUNG}
done

## Der `grep`-Filter (Dateiinhalte in Textdateien finden)

Wir wollen uns in diesem Abschnitt noch einen weiteren, sehr wichtigen, Filter ansehen, der sehr häufig in Pipelines verwendet wird. Es handelt sich um den den `grep`-Befehl. Genau wie der Begriff *`Google`-Suche* (oder *googeln*) inzwischen ein Synonym für *Webseiten mit bestimmten Inhalten finden* ist, ist `grep` unter `Unix`-Nutzern ein Synonym für *bestimmte Inhalte in Textdateien finden*.Der Name des Befehls ist eine Abkürzung für  *global/regular expression/print*. Die Folge `grep` wurde erstmals in den 1970er Jahren in dem Texteditor `vi` benutzt, um gewissen Textstellen zu finden. Hieraus hat sich später ein eigenständiges `Unix`-Programm entwickelt.

Wir behandeln hier lediglich die einfachste Form des Kommandos, das aber für den Anfang für Sie vollkommen ausreichend ist.

Wir hatten uns bereits in [Lektion 4](04_Shell_Pipelines_und_Filter.ipynb#Annika_Test_Beobachtungen) mit der Datei `Beobachtungs_Logdatei.txt` beschäftigt, welche Informationen über Annikas Beobachtungen enthält:

In [ ]:
cd ~/Bachelor_Arbeit/Beobachtungen/
head Beobachtungs_Logdatei.txt

Die für uns wichtigen Spalten sind hier die erste mit der Identifikation der Beobachtung, die zweite Spalte aus Beobachtungsfeld (`D1-D4`) und Beobachtungskonfiguration (`u`, `g`, `r`, `i` oder `z`) und die vierte Spalte, die den Beobachtungszeitpunkt (die Anzahl der Tage nach einem bestimmten Referenzdatum) angibt.

<div class="alert alert-success">
<b>Zum Mitnehmen</b>
<ul>
    <li> Die Kommandosubtitution <code>&#36;(kommando)</code> ersetzt ihr Vorkommen durch die Ausgabe von <code>kommando</code>. Dadurch ist es möglich, die Ausgabe eines Programms als Argumente eines anderen zu verwenden.</li>
</ul>
</div>